# Beyond Naïve Bayes: complex Bayesian Network Architectures
Build two or three Bayes networks of more complex architecture for (a smaller version of) this data set, increasing the number
of connections among the nodes. Construct one of them semi-manually (e.g use K2 algorithm and vary the maximum number of parents), and two others – using Weka’s algorithms for learning Bayes net construction (e.g. use TAN or Hill Climbing algorithms). Run the experiments described in item 5 on these new Bayes network architectures. Record, compare and analyse the outputs, in the light of the previous conclusions about the given data. 

In [1]:
import math
import numpy as np
import pandas as pd
#from pomegranate import *

import pgmpy.models
import pgmpy.inference
import networkx as nx
import pylab as plt

In [5]:
pixel_values = [x for x in range(0, 256)]



In [6]:
# Example

# Create a bayesian network
model = pgmpy.models.BayesianModel([('Guest', 'Monty'), 
                                    ('Prize', 'Monty')])

# Define conditional probability distributions (CPD)
# Probability of guest selecting door 0, 1 and 2
cpd_guest = pgmpy.factors.discrete.TabularCPD('Guest', 3, [[0.33], [0.33], [0.33]])
# Probability that the price is behind door 0, 1 and 2
cpd_prize = pgmpy.factors.discrete.TabularCPD('Prize', 3, [[0.33], [0.33], [0.33]])
# Probability that Monty selects a door (0, 1, 2), when we know which door the guest has selected and we know were the prize is
cpd_monty = pgmpy.factors.discrete.TabularCPD('Monty', 3, [[0, 0, 0, 0, 0.5, 1, 0, 1, 0.5], 
                                                           [0.5, 0, 1, 0, 0, 0, 1, 0, 0.5], 
                                                           [0.5, 1, 0, 1, 0.5, 0, 0, 0, 0]], 
                                              evidence=['Guest', 'Prize'], 
                                              evidence_card=[3, 3])

# Add CPDs to the network structure
model.add_cpds(cpd_guest, cpd_prize, cpd_monty)

# Check if the model is valid, throw an exception otherwise
model.check_model()
# Print probability distributions
print('Probability distribution, P(Guest)')
print(cpd_guest)
print()
print('Probability distribution, P(Price)')
print(cpd_prize)
print()
print('Joint probability distribution, P(Monty | Guest, Price)')
print(cpd_monty)
print()

# Plot the model
nx.draw(model, with_labels=True)
#plt.savefig('C:\\DATA\\Python-data\\bayesian-networks\\monty-hall.png')
plt.close()

# Perform variable elimination for inference
# Variable elimination (VE) is a an exact inference algorithm in bayesian networks
infer = pgmpy.inference.VariableElimination(model)

# Calculate probabilites for doors including prize, the guest has selected door 0 and Monty has selected door 2
posterior_probability = infer.query(['Prize'], evidence={'Guest': 0, 'Monty': 2})

# Print posterior probability
print('Posterior probability, Guest(0) and Monty(2)')
print(posterior_probability)
print()

Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]Probability distribution, P(Guest)
+----------+------+
| Guest(0) | 0.33 |
+----------+------+
| Guest(1) | 0.33 |
+----------+------+
| Guest(2) | 0.33 |
+----------+------+

Probability distribution, P(Price)
+----------+------+
| Prize(0) | 0.33 |
+----------+------+
| Prize(1) | 0.33 |
+----------+------+
| Prize(2) | 0.33 |
+----------+------+

Joint probability distribution, P(Monty | Guest, Price)
+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
| Guest    | Guest(0) | Guest(0) | Guest(0) | Guest(1) | Guest(1) | Guest(1) | Guest(2) | Guest(2) | Guest(2) |
+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
| Prize    | Prize(0) | Prize(1) | Prize(2) | Prize(0) | Prize(1) | Prize(2) | Prize(0) | Prize(1) | Prize(2) |
+----------+----------+----------+----------+----------+----------+---------

## Specific for 

In [2]:
from Scripts import helperfn as hf
from Scripts import pixelFinder as pf 
from Scripts.NaiveBayse import SamNaiveBayseGaussian as nbg
%load_ext autoreload
%autoreload 2


In [8]:
pixels = pf.get_top_pixels(5)

In [9]:
raw_data = pf.data_lists()

In [87]:
X, y = raw_data[1]

In [88]:
y.shape

(9690, 1)

In [89]:
five_pixels = np.take(X, pixels[1], axis=1)
five_pixels.shape

(9690, 5)

In [90]:
test = five_pixels[0:423]
result_col = y.copy()
y = y[0:423]



In [91]:
result_col.shape

(9690, 1)

In [92]:
pixels[0]

array([1714, 1762, 1761, 1743, 1694], dtype=int64)

In [15]:
model = pgmpy.models.BayesianModel([('1362', 'speed limit 20'), 
                                    ('982', 'speed limit 20'),
                                    ('1030', 'speed limit 20'),
                                    ('1315', 'speed limit 20'),
                                    ('1314', 'speed limit 20')])

In [16]:
def get_CPD(data, pixel_index, cardinality=256):
    #conditional probability distribution table for pixel at index
    entries = data.shape[0]
    calc = np.zeros(cardinality, dtype=int)

    for i in range(entries):
        val = data[i][pixel_index].astype(int)
        calc[val] += 1

    calc = calc/entries
    return calc[..., None].tolist()

def pixels_as_string(pixels):
    return list(map(str, pixels))



In [17]:
#cpd_1362 = pgmpy.factors.discrete.TabularCPD('1362', 256, get_CPD(test, 0))
#print('Probability distribution, P(1362)')
#print(cpd_1362)

In [18]:
cpd_labels = pixels_as_string(pixels[1])

independant_cpd_pixel_tables = [pgmpy.factors.discrete.TabularCPD(value, 256, get_CPD(test, index)) for index, value in enumerate(cpd_labels)]
independant_cpd_class_table = pgmpy.factors.discrete.TabularCPD('speed limit 20', 2, get_CPD(y, 0, cardinality=2))

In [19]:
print(independant_cpd_class_table)

+-------------------+----------+
| speed limit 20(0) | 0.496454 |
+-------------------+----------+
| speed limit 20(1) | 0.503546 |
+-------------------+----------+


In [21]:
#len(independant_cpd_tables)
#independant_cpd_tables[0].get_values()[55]

# need equal width binning for the grayscale values, try bin sizes of between 8 and 16

In [22]:
from itertools import combinations

from pgmpy.estimators import HillClimbSearch, ExhaustiveSearch
from pgmpy.estimators import K2Score

In [76]:
test = pd.DataFrame(five_pixels[0:423])
y = pd.DataFrame(y[0:423])


In [77]:
y.shape

(423, 1)

In [38]:
y.columns = ['y']
test = test.join(y)
test

,1362,982,1030,1315,1314,y
0,160.0,191.0,155.0,166.0,195.0,0
1,218.0,141.0,132.0,192.0,226.0,0
2,166.0,142.0,135.0,148.0,194.0,0
3,152.0,136.0,127.0,121.0,161.0,0
4,212.0,212.0,151.0,192.0,224.0,0
...,...,...,...,...,...,...
418,255.0,255.0,255.0,254.0,255.0,1
419,255.0,254.0,253.0,255.0,255.0,1
420,255.0,250.0,222.0,255.0,255.0,1
421,255.0,226.0,211.0,255.0,255.0,1


In [39]:
test = pd.DataFrame(test)
scoring_method = K2Score(data=test)
est = HillClimbSearch(data=test, scoring_method=scoring_method)
estimated_model = est.estimate(max_indegree=4, max_iter=int(1e4))

In [46]:
estimated_model.edges()

OutEdgeView([('1030', '982'), ('1314', '1315'), ('1314', '1362'), ('1314', '1030'), ('y', '1314'), ('y', '1362'), ('y', '1030')])

In [ ]:
learned edges = [('1030', '982'), ('1314', '1315'), ('1314', '1362'), ('1314', '1030'), ('y', '1314'), ('y', '1362'), ('y', '1030')]

In [44]:
from pgmpy.models import BayesianModel

In [48]:

model_struct = BayesianModel(estimated_model.edges)
model_struct.nodes()

NodeView(('1030', '982', '1314', '1315', '1362', 'y'))

In [53]:
from pgmpy.estimators import MaximumLikelihoodEstimator

mle.get_parameters()[:10]

[<TabularCPD representing P(1030:169 | 1314:182, y:2) at 0x1ee45c31be0>,
 <TabularCPD representing P(1314:182 | y:2) at 0x1ee45c32eb0>,
 <TabularCPD representing P(1315:177 | 1314:182) at 0x1ee45c314c0>,
 <TabularCPD representing P(1362:173 | 1314:182, y:2) at 0x1ee459f6730>,
 <TabularCPD representing P(982:174 | 1030:169) at 0x1ee45c31910>,
 <TabularCPD representing P(y:2) at 0x1ee45c31490>]

In [54]:
from pgmpy.estimators import BayesianEstimator

In [55]:
model_struct.fit(data=test, estimator=MaximumLikelihoodEstimator)


In [57]:
print(model_struct.get_cpds())


[<TabularCPD representing P(1030:169 | 1314:182, y:2) at 0x1ee46009310>, <TabularCPD representing P(1314:182 | y:2) at 0x1ee46009430>, <TabularCPD representing P(1315:177 | 1314:182) at 0x1ee46009760>, <TabularCPD representing P(1362:173 | 1314:182, y:2) at 0x1ee458e7340>, <TabularCPD representing P(982:174 | 1030:169) at 0x1ee46019670>, <TabularCPD representing P(y:2) at 0x1ee46019880>]


In [65]:
from pgmpy.inference import VariableElimination
five_pix_infer = VariableElimination(model_struct)

q = five_pix_infer.query(variables=['y'], evidence={'1362': 255})

Finding Elimination Order: : 100%|██████████| 4/4 [00:00<00:00, 444.45it/s]

Eliminating: 1314: 100%|██████████| 4/4 [00:00<00:00, 222.39it/s]


In [66]:
print(q)

+------+----------+
| y    |   phi(y) |
+======+==========+
| y(0) |   0.0000 |
+------+----------+
| y(1) |   1.0000 |
+------+----------+


In [93]:
test_data = five_pixels[423:424]
test_result = result_col[423:424]

test_data


,1362,982,1030,1315,1314
423,255.0,205.0,199.0,255.0,255.0


In [102]:
evidence = {'1362': 255, '982': 205, '1030':199, '1315':255, '1314': 255}

In [94]:
test_result

,y
423,1


In [105]:
q = five_pix_infer.query(variables=['y'], evidence={'1315':255})
print(q)

Finding Elimination Order: : 100%|██████████| 4/4 [00:00<00:00, 444.36it/s]

Eliminating: 1314: 100%|██████████| 4/4 [00:00<00:00, 190.48it/s]+------+----------+
| y    |   phi(y) |
+======+==========+
| y(0) |   0.0000 |
+------+----------+
| y(1) |   1.0000 |
+------+----------+



In [3]:
X, y = hf.get_ewb_data(0)

In [5]:
y.columns = ['y']
X = X.join(y)

In [10]:
pixels = pf.get_top_pixels(10)

In [11]:
ten_pixels = np.take(X, pixels[1], axis=1)
ten_pixels.shape

(9690, 10)

In [ ]:
ten_pixels.join(y)

In [38]:
ten_pixels

,1362,982,1030,1315,1314,1078,1409,1031,1410,934,y
0,light grey,light grey,semi light grey,light grey,very light grey,semi light grey,semi light grey,light grey,semi dark grey,very light grey,0
1,very light grey,semi light grey,semi light grey,very light grey,white,semi light grey,light grey,semi light grey,very light grey,light grey,0
2,light grey,semi light grey,semi light grey,semi light grey,very light grey,light grey,light grey,semi dark grey,semi light grey,light grey,0
3,semi light grey,semi light grey,semi dark grey,semi dark grey,light grey,semi dark grey,light grey,semi light grey,semi light grey,semi light grey,0
4,very light grey,very light grey,semi light grey,very light grey,white,semi light grey,very light grey,light grey,very light grey,white,0
...,...,...,...,...,...,...,...,...,...,...,...
9685,semi dark grey,semi dark grey,semi dark grey,dark grey,semi dark grey,semi dark grey,semi dark grey,semi dark grey,semi dark grey,semi dark grey,1
9686,semi dark grey,semi dark grey,semi dark grey,dark grey,semi dark grey,semi dark grey,semi dark grey,semi dark grey,semi dark grey,semi dark grey,1
9687,semi dark grey,semi dark grey,semi dark grey,very dark grey,semi dark grey,semi dark grey,semi dark grey,semi dark grey,dark grey,semi dark grey,1
9688,semi dark grey,semi dark grey,semi dark grey,dark grey,semi dark grey,semi dark grey,dark grey,semi dark grey,dark grey,dark grey,1


In [60]:
from itertools import combinations
from pgmpy.models import BayesianModel
from pgmpy.estimators import HillClimbSearch, ExhaustiveSearch
from pgmpy.estimators import K2Score
from pgmpy.inference import VariableElimination
from pgmpy.estimators import MaximumLikelihoodEstimator
import numpy as np

In [48]:
balanced = ten_pixels.iloc[:423]
balanced

,1362,982,1030,1315,1314,1078,1409,1031,1410,934,y
0,light grey,light grey,semi light grey,light grey,very light grey,semi light grey,semi light grey,light grey,semi dark grey,very light grey,0
1,very light grey,semi light grey,semi light grey,very light grey,white,semi light grey,light grey,semi light grey,very light grey,light grey,0
2,light grey,semi light grey,semi light grey,semi light grey,very light grey,light grey,light grey,semi dark grey,semi light grey,light grey,0
3,semi light grey,semi light grey,semi dark grey,semi dark grey,light grey,semi dark grey,light grey,semi light grey,semi light grey,semi light grey,0
4,very light grey,very light grey,semi light grey,very light grey,white,semi light grey,very light grey,light grey,very light grey,white,0
...,...,...,...,...,...,...,...,...,...,...,...
418,white,white,white,white,white,white,white,white,white,white,1
419,white,white,white,white,white,white,white,white,white,white,1
420,white,white,very light grey,white,white,very light grey,white,white,white,white,1
421,white,white,very light grey,white,white,white,white,white,white,white,1


In [49]:
scoring_method = K2Score(data=balanced)
est = HillClimbSearch(data=balanced, scoring_method=scoring_method)
estimated_model = est.estimate(max_indegree=4, max_iter=int(1e4))

In [50]:
model_struct = BayesianModel(estimated_model.edges)
model_struct.nodes()

NodeView(('1362', '1314', '1409', '1410', 'y', '982', '1030', '1078', '1315', '1031', '934'))

In [51]:
model_struct.fit(data=ten_pixels, estimator=MaximumLikelihoodEstimator)


In [55]:
model_struct.check_model()

True

In [52]:
ten_pix_infer = VariableElimination(model_struct)
q = ten_pix_infer.query(variables=['y'], evidence={'1362': 'dark grey', '982': 'dark grey', '1030': 'dark grey', '1315': 'very dark grey' , '1078': 'dark grey', '1409': 'dark grey', '1031': 'dark grey', '1410': 'dark grey', '934': 'dark grey'})
q

Eliminating: 1314: 100%|██████████| 1/1 [00:00<00:00, 500.27it/s]


<DiscreteFactor representing phi(y:2) at 0x28517600970>

In [53]:
ten_pixels_no_y = ten_pixels.drop('y', 1)
print(ten_pixels.iloc[0])
ev = ten_pixels_no_y.iloc[0].to_dict()

1362         light grey
982          light grey
1030    semi light grey
1315         light grey
1314    very light grey
1078    semi light grey
1409    semi light grey
1031         light grey
1410     semi dark grey
934     very light grey
y                     0
Name: 0, dtype: object


In [61]:
q = ten_pix_infer.query(variables=['y'], evidence=ev)
print(q)
np.argmax(q.values)

Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]+------+----------+
| y    |   phi(y) |
+======+==========+
| y(0) |   0.0652 |
+------+----------+
| y(1) |   0.9348 |
+------+----------+



1

In [74]:
def score_model(model, test_data, labels, result_label='y', result_cardinality=2):
    good_pred = 0
    for i in range(test_data.shape[0]):
        ev = test_data.iloc[i].to_dict()
        q = model.query(variables=[result_label], evidence=ev)
        pred = np.argmax(q)
        if pred == labels.iloc[i].values[0]:
            good_pred += 1
    return good_pred / test_data.shape[0] 


In [79]:
score = score_model(ten_pix_infer, ten_pixels_no_y.iloc[:423], y.iloc[:423])
score

 [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
{'1362': 'light grey', '982': 'semi dark grey', '1030': 'semi dark grey', '1315': 'very light grey', '1314': 'very light grey', '1078': 'semi light grey', '1409': 'light grey', '1031': 'semi dark grey', '1410': 'semi light grey', '934': 'semi light grey'}
{'1362': 'light grey', '982': 'semi light grey', '1030': 'semi dark grey', '1315': 'very light grey', '1314': 'very light grey', '1078': 'semi dark grey', '1409': 'semi light grey', '1031': 'semi dark grey', '1410': 'light grey', '934': 'light grey'}
{'1362': 'light grey', '982': 'semi dark grey', '1030': 'dark grey', '1315': 'light grey', '1314': 'light grey', '1078':

0.49645390070921985

In [76]:
ten_pixels_no_y.shape

(9690, 10)

In [73]:
y.iloc[10].values[0]

0

# To go in submission notebook

In [1]:
from Scripts import helperfn as hf
from Scripts import pixelFinder as pf 
from Scripts.NaiveBayse import NaiveBayseGaussian as nbg
from Scripts import bayseNet as bn
from sklearn.model_selection import train_test_split
%load_ext autoreload
%autoreload 2

import math
import numpy as np
import pandas as pd
from pgmpy.estimators import BayesianEstimator, HillClimbSearch, ExhaustiveSearch, K2Score, MaximumLikelihoodEstimator, BicScore, BDeuScore



## Preprocess data
### Get equal width binned data and labels

In [2]:
X, y = hf.get_ewb_data(4)

#### Select most predicitve pixels

In [4]:
pixels = pf.get_top_pixels(15)
# pixels is list of best pixels for all datasets: pixels[0] is all classes | pixels[1] is speed limit 20...
x_pixels = np.take(X, pixels[5], axis=1)
x_pixels.shape

(9690, 15)

#### Select a balanced class distrubution from the data and append labels to data

In [5]:
y.columns = ['y']
x_bal_pixels, y_pixels = hf.balance_by_class(x_pixels, y)
X_train, x_test, y_train, y_test = train_test_split(x_bal_pixels, y_pixels, random_state=0, test_size=0.2)
X_pixels = X_train.join(y_train)

## Build model
#### Learn edges with K2 parents = 4, HillClimbing algorithm

In [6]:
hillClimb = bn.estimate_model_edges(X_pixels)

---- Beginning edge estimator ----
---- Done ----


#### Learn model parameters using BDeu as the estimator

In [7]:
hillClimb.edges

OutEdgeView([('983', '935'), ('983', '982'), ('983', 'y'), ('1031', '983'), ('1031', 'y'), ('1030', '982'), ('1030', '1031'), ('982', 'y'), ('935', 'y'), ('2139', '2138'), ('2138', '2186'), ('2138', '1030'), ('2140', '2139'), ('2095', '2094'), ('2096', '2095'), ('2096', '2141'), ('2141', '2140'), ('2050', '2097'), ('2094', '2141'), ('2097', '2096')])

In [13]:
model = bn.model_with_params(X_pixels, hillClimb, estimator=BayesianEstimator, prior_type="K2")

['983', '935', '982', 'y', '1031', '1030', '2139', '2138', '2186', '2140', '2095', '2094', '2096', '2141', '2050', '2097']


In [14]:
type(model)

pgmpy.models.BayesianModel.BayesianModel

In [15]:
infer = bn.get_inference_model(model)

## Score the model

In [16]:
score = bn.score_model(infer, x_test, y_test)

: 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]looping
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
looping
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]looping
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
looping
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]looping
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
looping
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
looping
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Eliminati

In [12]:
print(score)

0.5164141414141414
